# Анализ результатов А/В тестирования изменения дизайна сайта онлайн магазина

Исполнитель: Никитина Александра  
Email: nikilinalexa@gmail.com  
Telegram: https://t.me/alexandrakilina

## Цели и задачи проекта

Оценить корректность и проанализировать результаты A/B-тестирования изменений дизайна сайта с целью увеличения конверсии зарегистрированных пользователей в покупателе. Проверить, что конверсия зарегистрированных пользователей в покупателей увеличилась как минимум на 3 процентных пункта.

Гипотезы:
* H0: нулевая гипотеза, изменение дизайна интерфейса не изменило пользовательскую активность, число продаж осталось неизменным или выросло менее, чем на 3 процентных пункта.
* Н1: изменение дизайна интерфейса увеличило пользовательскую активность, как минимум на 3% пункта.

Параметры для обеспечения мощности выборки:

* базовый показатель конверсии — 30%,
* мощность теста — 80%,
* достоверность теста — 95%.


**Содержимое проекта**:
1. Загрузка данных и знакомство с ними
2. Оценка корректности проведения А/В-теста: оценка наличия пересечений с конкурирующим тестм, независимости выборок, мощности выборок, анализ данных о пользовательской активности
3. Оценка результатов А/В-тестирования
4. Результаты и выводы

## Загрузка данных и знакомство с ними

Параметрый теста:
* Название теста: `interface_eu_test`
* Группы: A - контрольная, В - тестовая

Названия файлов:  
* 'название файла скрыто'.csv - таблица участников теста, включает в себя идентификатор пользователя, тип группы, название теста, устройство, с которого происходила регистрация.
* 'название файла скрыто'.zip - архив с .csv файлом, в котором собраты события за 2020 года. Включает в себя идентификатор пользователя, дату и время события, тип события, детали события (стоимость привлечения клиента / стоимость покупки / зона).

*Внимание, часть информации по загрузке и знакомству данных скрыта*

In [1]:
# импортируем библиотеку pandas
import pandas as pd

# установим библиотеку jupyter-black для автоматического форматирования кода
!pip install jupyter-black

# Импортируем библиотеку jupyter_black
import jupyter_black

# Импортируем библиотеки для расчета размера выборки
from statsmodels.stats.power import NormalIndPower
from statsmodels.stats.proportion import proportion_effectsize

# Импортируем библиотеку для проведения z-теста пропорций
from statsmodels.stats.proportion import proportions_ztest

In [2]:
# Выгрузим данные в датафреймы participants и events
participants = pd.read_csv('название файла скрыто.csv')
events = pd.read_csv('название файла скрыто.zip',
                     parse_dates=['event_dt'], low_memory=False)

### Знакомство с датафреймом `participants`

In [3]:
# Выведем информацию о датафрейме participants
participants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14525 entries, 0 to 14524
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  14525 non-null  object
 1   group    14525 non-null  object
 2   ab_test  14525 non-null  object
 3   device   14525 non-null  object
dtypes: object(4)
memory usage: 454.0+ KB


In [4]:
# Выведем первые 5 строк датафрейма
#participants.head()

In [5]:
# Проверим наличие дубликатов
participants.duplicated().sum()

0

In [6]:
# Выведем уникальные значения для столбца ab_test
#display(participants["ab_test"].unique())

**Вывод:** 

В датафрейме `participants`: 
* отсутствуют ненулевые значения
* присутствуют данные для двух A/B-тестов, данные необходимо будет отфильтровать по целевому тесту `interface_eu_test`
* явные дубликаты отсутствуют

### Знакомство с датафреймом `events`

In [7]:
# Выведем информацию о датафрейме events
#events.info()

In [8]:
# Выведем первые 5 строк датафрейма
#events.head()

In [9]:
# Проверим наличие дубликатов
participants.duplicated().sum()

0

In [10]:
# Выведем на экран типы событий из столбца event_name
# events['event_name'].unique()

In [11]:
# Посмотрим, как будут выглядить данные для одного пользователя
# events[events['user_id'] == 'CCBE9E7E99F94A08']

**Вывод:** 
В датафрейме `events`:
* Выявлены пропуски в столбце details
* Явные дубликаты отсутствуют
* Выведены типы событий столбца event_name, в дальнейшем для анализа нам понадобятся события, связанные с датой регистрации 'registration' и с датой совершения пользователем события

## Оценка корректности проведения A/B теста

   Для пользователей, участвующих в тесте, необходимо проверить соответствие требованиям технического задания, равномерность распределения пользователей по группам и отсутствие пересечений с конкурирующим тестом.

### Проверка наличия пересечений с конкурирующим тестом

In [12]:
# Проверим, пересекаются ли пользователи в группах B тестов interface_eu_test и recommender_system_test
group_b_rec = participants[(participants['ab_test'] == 'recommender_system_test')&(participants['group'] == 'B')]['user_id']
group_b_eu = participants[
    (participants['ab_test'] == 'interface_eu_test') & (participants['group'] == 'B')
]['user_id']

intersection_b_tests = list(set(group_b_eu) & set(group_b_rec))

print(f'Количество пользователей, участвовавших в конкрурирующих тестах: {len(intersection_b_tests)}')

Количество пользователей, участвовавших в конкрурирующих тестах: 116


*Промежуточный вывод:*

Среди пользователей групп В тестов `interface_eu_test` и `recommender_system_test` есть те пользователи, которые участвовали в обоих тестах, что является нарушением А/В теста, так как пользователь не  должен участвовать в двух тестах одновременно.

In [13]:
# Так как по ТЗ необходимо проанализировать данные теста interface_eu_test, отфильтруем датафрейм
participants_interface_eu_test = participants[participants['ab_test'] == 'interface_eu_test']

In [14]:
# Удалим тех пользователей, который участвуют в обоих тестах одновременно
participants_interface_filtered = participants_interface_eu_test[~participants_interface_eu_test['user_id'].isin(intersection_b_tests)]

**Вывод:**

Среди пользователей групп В тестов `interface_eu_test` и `recommender_system_test` выявлены пользователи, принимавшие участие в обоих тестах, что является нарушением А/В теста, так как пользователь не  должен участвовать в двух тестах одновременно.

Сформирован отфильтрованный датафрейм `participants_interface_filtered`, из которого исключены пользователи, которые входили одновременно в оба А/В теста.

### Оценка распределения пользователей между группами А и В

Оценим равномерность распределения пользователей между группами А и В в датафрейме, отфильтрованному по целевому тесту `interface_eu_test`

In [15]:
# Подсчитаем количество уникальных пользователей в каждой из групп теста interface_eu_test
user_counts = participants_interface_eu_test.groupby('group')['user_id'].nunique().reset_index()

# Извлекаем значения количества уникальных пользователей в каждой из групп
count_a = user_counts[user_counts['group']=='A']['user_id'].values[0]
count_b = user_counts[user_counts['group']=='B']['user_id'].values[0]

# Посчитаем процентную разницу в кол-ве пользователей в группах А и В
p = round(100*abs(count_a - count_b) / count_a, 2)

print(f"Процентная разница в количестве пользователей в группах А и В теста interface_eu_test: {p}%")

Процентная разница в количестве пользователей в группах А и В теста interface_eu_test: 1.56%


**Вывод:** Разница в количестве пользователей групп А и В тесте `interface_eu_test` составила 1.56%, можно заключить, что пользователи распределены равномерно

### Анализ данных о пользовательской активности

Отфильтруем датафрейм `events`, удалив события, связанные с пользователями, которые принимали участие одновременно в двух тестах.

In [16]:
# Удалим события, связанные с пользователями, участвующими более, чем в одном тесте
events_filtered = events[~events['user_id'].isin(intersection_b_tests)]

Рассчитаем время совершения события пользователем после регистрации и оставим только те события, которые были выполнены в течение первых семи дней с момента регистрации.

In [17]:
# Для каждого пользователя отберем дату регистрации
users_registration = events_filtered[events_filtered['event_name'] == 'registration'][['user_id', 'event_dt']].drop_duplicates()

# Переименуем поле для удобства
users_registration = users_registration.rename(columns={'event_dt': 'registration_dt'})

In [18]:
# Создадим датафрейм, в котором к датафрейму events присоединим дату регистрации пользователя
events_with_reg = events_filtered.merge(users_registration, on = 'user_id', how = 'inner')

# Рассчитаем разницу во времени
events_with_reg['lifetime'] = (events_with_reg['event_dt'] - events_with_reg['registration_dt']).dt.days

# Отфильтруем события, у которых разница во времени составляет менее 7 дней
events_with_reg = events_with_reg[events_with_reg['lifetime'] < 7]

Проведем оценку достаточности выборки со следующими заданными параметрами:
* базовый показатель конверсии — 30%
* мощность теста — 80%
* достоверность теста — 95%

In [19]:
# Задаем исходные параметры
alpha = 0.05
power = 0.8
# Базовая конверсия
p1 = 0.30  
# Прирост на 3 процентных пункта
p2 = 0.33 
effect_size = proportion_effectsize(p1, p2)

# Расчёт мощности
analysis = NormalIndPower()
sample_size = analysis.solve_power(
    effect_size=effect_size,
    power=power,
    alpha=alpha,
    ratio=1
)

print(f"Необходимый размер выборки на каждую группу: {int(sample_size)} пользователей")

Необходимый размер выборки на каждую группу: 3761 пользователей


Для каждой группы рассчитаем количество посетителей, сделавших покупку, и общее количество посетителей.

In [20]:
# Отфильтруем события, относящиеся к interface_eu_test, чтобы анализировать только тех, кто участвует в целевом тесте
events_interface_eu_test = events_with_reg[
    events_with_reg['user_id'].isin(participants_interface_filtered['user_id'])
]

# Оставляем только события покупки (event_name == 'purchase')
purchases = events_interface_eu_test[events_interface_eu_test['event_name'] == 'purchase']

# Присоединяем информацию о группах к покупкам, в какой группе находится пользователя
purchases_with_groups = purchases.merge(
    participants_interface_filtered[['user_id', 'group']],
    on='user_id',
    how='inner'
)

# Считаем кол-во уникальных покупателей по группам
buyers_by_group = purchases_with_groups.groupby('group')['user_id'].nunique().reset_index()
buyers_by_group.columns = ['group', 'buyers']

# Считаем общее количество участников теста в каждой группе (знаменатель для расчета конверсии)
total_users_by_group = participants_interface_filtered.groupby('group')['user_id'].nunique().reset_index()
total_users_by_group.columns = ['group', 'total_users']

# Объединяем таблицы и считаем конверсию (покупатели / все участники группы)
summary = buyers_by_group.merge(total_users_by_group, on='group')
summary['conversion_rate'] = round(summary['buyers'] / summary['total_users'], 4)

print(summary)

  group  buyers  total_users  conversion_rate
0     A    1480         5383           0.2749
1     B    1579         5351           0.2951


In [21]:
# Рассчитаем разницу конверсии
conversion_rate_a = summary[summary['group'] == 'A']['conversion_rate'].values[0]
conversion_rate_b = summary[summary['group'] == 'B']['conversion_rate'].values[0]
difference = round(100*(conversion_rate_b - conversion_rate_a), 2)
print(f'В группе В конверсия пользователя в покупку увеличилась на {difference}% по сравнению с контрольной группой')

В группе В конверсия пользователя в покупку увеличилась на 2.02% по сравнению с контрольной группой


**Вывод**

Общее количество пользователей в каждой группе составило не менее 5383 пользователя, что больше необходимого (3761 пользователей). 

Пользовательская активность была выявлена и в группе А, и в группе В, конверсия составила 0.2749 для группы А и 0.2951 для группы В.

Конверсия в группе В увеличилась на 2.02% по сравнению с конверсией контрольной группы.

Таким образом, завершенный тест включает в себя достаточное количество данных для того, чтобы провести оценку результатов А/В тестирования, хотя целевое увеличение конверсии не было достигнуто

## Оценка результатов A/B-тестирования

**Гипотезы:**

H0: нулевая гипотеза, изменение дизайна интерфейса не изменило пользовательскую активность, число продаж осталось неизменным или выросло менее, чем на 3 процентных пункта.

Н1: изменение дизайна интерфейса увеличило пользовательскую активность, как минимум на 3% пункта.

Таким образом, нужно проверить односторонню гипотезу на увеличение. Для этого можно использовать z-тест для сравнения пропорций с использованием функции proportions_ztest из пакета statsmodels.stats.proportion 

In [22]:
# Берем значения по размерам выборок
n_a = int(summary[summary['group']=='A']['total_users'])
n_b = int(summary[summary['group']=='B']['total_users'])

# Берем значения количества покупателей
m_a = int(summary[summary['group']=='A']['buyers'])
m_b = int(summary[summary['group']=='B']['buyers'])


# Уровень значимости, на котором проверяем гипотезу
alpha = 0.05 

# Расчет статистического теста
z_stat, p_value = proportions_ztest(
    [m_a, m_b],
    [n_a, n_b],
    alternative='larger', 
    value=0.03
    
)
print(f"z-статистика: {round(z_stat, 2)}, p-value: {round(p_value, 4)}")

if p_value > alpha:
    print(f'pvalue={p_value} > {alpha}')
    print('Нулевая гипотеза находит подтверждение, прирост менее 3 п.п. или его нет.')
else:
    print(f'pvalue={p_value} < {alpha}')
    print('Нулевая гипотеза не находит подтверждения, конверсия увеличилась минимум на 3 п.п.')

z-статистика: -5.75, p-value: 1.0
pvalue=0.9999999956568034 > 0.05
Нулевая гипотеза находит подтверждение, прирост менее 3 п.п. или его нет.


## Результаты и выводы

По итогам результатов анализа A/B-теста о необходимости изменения дизайна сайта с целью увеличения конверсии пользователя в покупку было выявлено, что введение нового дизайна не привело к изменению целевой метрики более, чем на 3 процентных пункта по сравнению с контрольной группой. 

В ходе анализа результатов А/В-теста выло выполнено следующее: 
* Отобраны данные для теста `interface_eu_test`.
* Проверено, что группы А и В сопоставимы по размеру, размеры выборок превышают теоретически вычисленный порог (3761 пользователей), выборки являются независимыми. 
* Отобраны события, для которых покупка была совершена в течение 7 дней после регистрации (горизонт анализа - 7 дней). 
* Рассчитана `conversion rate` каждой группы, конверсия для группы В была на 2.02% больше конверсии для группы А.
* Анализ результатов теста был проведен с использованием z-теста для сравнения пропорций.
* p-value составило 1.00, что больше уровня значимости 0.05. Таким образом нулевая гипотеза находит подтверждение, что говорит о том, что изменение дизайна не привело к статистически значимому увеличению конверсии на 3 процентных пункта.

Введение нового дизайна не рекомендуется.